In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
6,model_9_9_14,0.879551,0.820616,0.768168,0.809062,0.022533,0.029437,0.031941,0.030615,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
8,model_9_9_13,0.878746,0.826851,0.780060,0.817243,0.022684,0.028414,0.030302,0.029303,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
10,model_9_9_12,0.877802,0.833047,0.791791,0.825346,0.022860,0.027397,0.028686,0.028004,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
12,model_9_9_11,0.876698,0.839453,0.802923,0.833322,0.023067,0.026346,0.027152,0.026725,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
14,model_9_9_10,0.875381,0.846033,0.813134,0.841015,0.023313,0.025266,0.025745,0.025492,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
15,model_9_9_9,0.873852,0.852636,0.822574,0.848415,0.023599,0.024182,0.024445,0.024305,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
16,model_9_9_8,0.872126,0.859315,0.831400,0.855602,0.023922,0.023086,0.023229,0.023153,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
17,model_9_9_7,0.870162,0.865965,0.839505,0.862481,0.024290,0.021995,0.022112,0.022050,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
18,model_9_9_6,0.867965,0.873089,0.846332,0.869100,0.024701,0.020826,0.021172,0.020989,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
19,model_9_9_5,0.865519,0.880237,0.852329,0.875400,0.025158,0.019653,0.020345,0.019978,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
